# Caching Data

Spark offers the possibility to cache data, which means that it tries to keep (intermediate) results either in memory or on disk. This can be very helpful in iterative algorithms or interactive analysis, where you want to prevent that the same processing steps are performed over and over again.

### Approach to Caching
Instead of performing timings of individual executions, we use the `explain()` method again to see how output changes with cached intermediate results.

### Weather Example
We will again use the weather example to understand how caching works.

## Create or Reuse Spark Session

In [1]:
from pyspark.sql import SparkSession

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","24G") \
        .getOrCreate()

spark

In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# 1. Load Data

First we load the weather data, which consists of the measurement data and some station metadata.

In [3]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

Measurements are stored in multiple directories (one per year)

In [4]:
import pyspark.sql.functions as f

# Union all years together
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", f.lit(2003))    

### Extract Measurements

Measurements were stored in a proprietary text based format, with some values at fixed positions. We need to extract these values with a simple `SELECT` statement.

In [5]:
weather = raw_weather.select(
    f.col("year"),
    f.substring(f.col("value"),5,6).alias("usaf"),
    f.substring(f.col("value"),11,5).alias("wban"),
    f.substring(f.col("value"),16,8).alias("date"),
    f.substring(f.col("value"),24,4).alias("time"),
    f.substring(f.col("value"),42,5).alias("report_type"),
    f.substring(f.col("value"),61,3).alias("wind_direction"),
    f.substring(f.col("value"),64,1).alias("wind_direction_qual"),
    f.substring(f.col("value"),65,1).alias("wind_observation"),
    (f.substring(f.col("value"),66,4).cast("float") / f.lit(10.0)).alias("wind_speed"),
    f.substring(f.col("value"),70,1).alias("wind_speed_qual"),
    (f.substring(f.col("value"),88,5).cast("float") / f.lit(10.0)).alias("air_temperature"),
    f.substring(f.col("value"),93,1).alias("air_temperature_qual")
)
    
weather.limit(10).toPandas()

,year,usaf,wban,date,time,report_type,wind_direction,wind_direction_qual,wind_observation,wind_speed,wind_speed_qual,air_temperature,air_temperature_qual
0,2003,703160,25624,20030101,0000,SY-MT,010,5,N,5.2,5,-0.6,5
1,2003,703160,25624,20030101,0017,FM-16,020,1,N,4.6,1,-2.0,1
2,2003,703160,25624,20030101,0053,FM-15,010,5,N,5.2,5,-2.8,5
3,2003,703160,25624,20030101,0100,NSRDB,999,9,9,999.9,9,999.9,9
4,2003,703160,25624,20030101,0153,FM-15,010,5,N,6.2,5,-2.2,5
5,2003,703160,25624,20030101,0200,NSRDB,999,9,9,999.9,9,999.9,9
6,2003,703160,25624,20030101,0253,FM-15,010,5,N,7.2,5,-3.3,5
7,2003,703160,25624,20030101,0300,NSRDB,999,9,9,999.9,9,999.9,9
8,2003,703160,25624,20030101,0353,FM-15,020,5,N,6.2,5,-1.1,5
9,2003,703160,25624,20030101,0400,NSRDB,999,9,9,999.9,9,999.9,9


## 1.2 Load Station Metadata

We also need to load the weather station meta data containing information about the geo location, country etc of individual weather stations.

In [6]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# Display first 10 records    
stations.limit(10).toPandas()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,None,None,None,+00.000,+000.000,+7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,None,None,+00.000,+000.000,+7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,None,None,+00.000,+000.000,+7070.0,20140923,20150926
3,008260,99999,WXPOD8270,None,None,None,+00.000,+000.000,+0000.0,20050101,20100920
4,008268,99999,WXPOD8278,AF,None,None,+32.950,+065.567,+1156.7,20100519,20120323
5,008307,99999,WXPOD 8318,AF,None,None,+00.000,+000.000,+8318.0,20100421,20100421
6,008411,99999,XM20,None,None,None,None,None,None,20160217,20160217
7,008414,99999,XM18,None,None,None,None,None,None,20160216,20160217
8,008415,99999,XM21,None,None,None,None,None,None,20160217,20160217
9,008418,99999,XM24,None,None,None,None,None,None,20160217,20160217


# 2 Caching Data

For analysing the impact of cachign data, we will use a slightly simplified variant of the weather analysis (only temperature will be aggregated). We will change the execution by caching intermediate results and watch how execution plans change.

## 2.1 Original Execution Plan

First let's have the execution plans of the original query as our reference.

In [7]:
joined_weather = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
aggregates = joined_weather.groupBy(joined_weather.CTRY, joined_weather.year).agg(
        f.min(f.when(joined_weather.air_temperature_qual == f.lit(1), joined_weather.air_temperature)).alias('min_temp'),
        f.max(f.when(joined_weather.air_temperature_qual == f.lit(1), joined_weather.air_temperature)).alias('max_temp')
    )

In [8]:
result = joined_weather.join(aggregates, ["ctry", "year"])

result.explain()

== Physical Plan ==
*(8) Project [CTRY#50, 2003 AS year#2, usaf#5, wban#6, date#7, time#8, report_type#9, wind_direction#10, wind_direction_qual#11, wind_observation#12, wind_speed#13, wind_speed_qual#14, air_temperature#15, air_temperature_qual#16, USAF#47, WBAN#48, STATION NAME#49, STATE#51, ICAO#52, LAT#53, LON#54, ELEV(M)#55, BEGIN#56, END#57, ... 2 more fields]
+- *(8) SortMergeJoin [CTRY#50], [CTRY#188], Inner
   :- *(3) Sort [CTRY#50 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(CTRY#50, 200), true, [id=#123]
   :     +- *(2) BroadcastHashJoin [usaf#5, wban#6], [USAF#47, WBAN#48], Inner, BuildRight
   :        :- *(2) Project [substring(value#0, 5, 6) AS usaf#5, substring(value#0, 11, 5) AS wban#6, substring(value#0, 16, 8) AS date#7, substring(value#0, 24, 4) AS time#8, substring(value#0, 42, 5) AS report_type#9, substring(value#0, 61, 3) AS wind_direction#10, substring(value#0, 64, 1) AS wind_direction_qual#11, substring(value#0, 65, 1) AS wind_observation#12, 

## 2.2 Caching Weather

First let us simply cache the joined input DataFrame.

In [9]:
joined_weather.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string, USAF: string, WBAN: string, STATION NAME: string, CTRY: string, STATE: string, ICAO: string, LAT: string, LON: string, ELEV(M): string, BEGIN: string, END: string]

### Forcing physical caching

The `cache()` method again works lazily and only marks the DataFrame to be cached. The physical cache itself will only take place once the elements are evaluated. A common and easy way to enforce this is to call a `count()` on the to-be cached DataFrame.

In [10]:
joined_weather.count()

1807253

### Execution Plan with Cache

Now let us have a look at the execution plan with the cache for the `weather` DataFrame enabled.

In [11]:
aggregates = joined_weather.groupBy(joined_weather.CTRY, joined_weather.year).agg(
        f.min(f.when(joined_weather.air_temperature_qual == f.lit(1), joined_weather.air_temperature)).alias('min_temp'),
        f.max(f.when(joined_weather.air_temperature_qual == f.lit(1), joined_weather.air_temperature)).alias('max_temp')
    )

result = joined_weather.join(f.broadcast(aggregates), ["ctry", "year"])

result.explain()

== Physical Plan ==
*(3) Project [CTRY#50, year#2, usaf#5, wban#6, date#7, time#8, report_type#9, wind_direction#10, wind_direction_qual#11, wind_observation#12, wind_speed#13, wind_speed_qual#14, air_temperature#15, air_temperature_qual#16, USAF#47, WBAN#48, STATION NAME#49, STATE#51, ICAO#52, LAT#53, LON#54, ELEV(M)#55, BEGIN#56, END#57, ... 2 more fields]
+- *(3) BroadcastHashJoin [CTRY#50, year#2], [CTRY#784, year#780], Inner, BuildRight
   :- *(3) Filter isnotnull(CTRY#50)
   :  +- InMemoryTableScan [year#2, usaf#5, wban#6, date#7, time#8, report_type#9, wind_direction#10, wind_direction_qual#11, wind_observation#12, wind_speed#13, wind_speed_qual#14, air_temperature#15, air_temperature_qual#16, USAF#47, WBAN#48, STATION NAME#49, CTRY#50, STATE#51, ICAO#52, LAT#53, LON#54, ELEV(M)#55, BEGIN#56, END#57], [isnotnull(CTRY#50)]
   :        +- InMemoryRelation [year#2, usaf#5, wban#6, date#7, time#8, report_type#9, wind_direction#10, wind_direction_qual#11, wind_observation#12, wind_sp

In [12]:
result.limit(10).toPandas()

,CTRY,year,usaf,wban,date,time,report_type,wind_direction,wind_direction_qual,wind_observation,...,STATION NAME,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,min_temp,max_temp
0,US,2003,703160,25624,20030101,0000,SY-MT,010,5,N,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
1,US,2003,703160,25624,20030101,0017,FM-16,020,1,N,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
2,US,2003,703160,25624,20030101,0053,FM-15,010,5,N,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
3,US,2003,703160,25624,20030101,0100,NSRDB,999,9,9,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
4,US,2003,703160,25624,20030101,0153,FM-15,010,5,N,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
5,US,2003,703160,25624,20030101,0200,NSRDB,999,9,9,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
6,US,2003,703160,25624,20030101,0253,FM-15,010,5,N,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
7,US,2003,703160,25624,20030101,0300,NSRDB,999,9,9,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
8,US,2003,703160,25624,20030101,0353,FM-15,020,5,N,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0
9,US,2003,703160,25624,20030101,0400,NSRDB,999,9,9,...,COLD BAY AIRPORT,AK,PACD,+55.221,-162.732,+0023.8,19420304,20200205,-44.0,41.0


### Remarks

Although the data is already cached, the execution plan still contains all steps. But the caching step won't be executed any more (since data is already cached), it is only mentioned here for completenss of the plan. We will see in the web interface.

The cache itself is presented as two steps in the execution plan:
* Creating the cache (InMemoryRelation)
* Using the cache (InMemoryTableScan)

If you look closely at the execution plans and compare these to the original uncached plan, you will notice that certain optimizations are not performed any more:
* Cache contains ALL columns of the weather DataFrame, although only a subset is required.
* Filter operation of JOIN is performed part of caching.

Caching is an optimization barrier. This means that Spark can only optimize plans before building the cache and plans after using the cache. No optimization is possible that spans building and using the cache. The idea simply is that the DataFrame should be cached exactly how it was specified without any column truncating or record filtering in place which appears after the cache.

## 2.2 Uncaching Data

Caches occupy resources (memory and/or disk). Once you do not need the cache any more, you'd probably like to free up the resources again. This is easily possible with the `unpersist()` method.

In [15]:
joined_weather.unpersist()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string, USAF: string, WBAN: string, STATION NAME: string, CTRY: string, STATE: string, ICAO: string, LAT: string, LON: string, ELEV(M): string, BEGIN: string, END: string]

### Exeuction plan after unpersist

Now we'd expect to have the original execution plan again.

In [18]:
result = joined_weather.groupBy(joined_weather.CTRY, joined_weather.year).agg(
        f.min(f.when(joined_weather.air_temperature_qual == f.lit(1), joined_weather.air_temperature)).alias('min_temp'),
        f.max(f.when(joined_weather.air_temperature_qual == f.lit(1), joined_weather.air_temperature)).alias('max_temp')
    )

result.explain(False)

== Physical Plan ==
*(6) HashAggregate(keys=[CTRY#52, 2003#2431], functions=[min(CASE WHEN (cast(air_temperature_qual#18 as int) = 1) THEN air_temperature#17 END), max(CASE WHEN (cast(air_temperature_qual#18 as int) = 1) THEN air_temperature#17 END)])
+- Exchange hashpartitioning(CTRY#52, 2003#2431, 200), true, [id=#476]
   +- *(5) HashAggregate(keys=[CTRY#52, 2003 AS 2003#2431], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#18 as int) = 1) THEN air_temperature#17 END), partial_max(CASE WHEN (cast(air_temperature_qual#18 as int) = 1) THEN air_temperature#17 END)])
      +- *(5) Project [air_temperature#17, air_temperature_qual#18, CTRY#52]
         +- *(5) SortMergeJoin [usaf#7, wban#8], [USAF#49, WBAN#50], Inner
            :- *(2) Sort [usaf#7 ASC NULLS FIRST, wban#8 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(usaf#7, wban#8, 200), true, [id=#458]
            :     +- *(1) Project [substring(value#2, 5, 6) AS usaf#7, substring(value#2, 11, 5) 

### Remarks

As you see in the execution plan, the cache has been removed now and the plan equals to the original one before we started caching data.

# 3 Cache Levels

Spark supports different levels of cache (memory, disk and a combination). These can be specified explicitly if you use `persist()` instead of `cache()`. Cache actually is a shortcut for `persist(MEMORY_AND_DISK)`.

In [20]:
from pyspark.storagelevel import StorageLevel

weather.persist(StorageLevel.MEMORY_ONLY)
weather.persist(StorageLevel.DISK_ONLY)
weather.persist(StorageLevel.MEMORY_AND_DISK)

weather.persist(StorageLevel.MEMORY_ONLY_2)
weather.persist(StorageLevel.DISK_ONLY_2)
weather.persist(StorageLevel.MEMORY_AND_DISK_2)


DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

### Cache level explanation

* `MEMORY_ONLY` - stores all records directly in memory
* `DISK_ONLY` - stores all records serialized on disk
* `MEMORY_AND_DISK` - stores all records first in memory and spills onto disk when no space is left in memory
* `..._2` - stores caches on two nodes instead of one for additional redundancy